In [22]:
import shutil

shutil.rmtree('/content/recordings/download')

In [23]:
import pandas as pd
import zipfile
import numpy as np
from pathlib import Path 

In [24]:
with zipfile.ZipFile('/content/download.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/recordings/')

## GENEROWANIE WYNIKÓW DLA PRÓB

In [25]:
import os

rootdir = os.fsencode('/content/recordings/download/')

recordings = []
manual = []
i = 0
j = 0

for subdir, dirs, files in os.walk(rootdir):
  if "recognized_words" in str(subdir): 
    for file in os.listdir(subdir):
      filename = os.fsdecode(file)
      if filename.endswith("words.csv"):
        recordings.append([os.path.join(str(subdir.decode("utf-8")), filename), i])
        i = i + 1
      if filename.endswith("tt.csv") or filename.endswith("tt2.csv"):
        manual.append([os.path.join(str(subdir.decode("utf-8")), filename), j])
        j = j + 1

In [ ]:
recordings

In [ ]:
manual

In [28]:
def check_comp(data_frame):
  check_compatibility = []
  for idx, m in data_frame.iterrows():
    if m['MANUAL'] == m['MODEL']:
      check_compatibility.append(1)
    else:
      check_compatibility.append(0)
  return check_compatibility

In [29]:
gt = pd.DataFrame.from_records(manual)
model_rec = pd.DataFrame.from_records(recordings)
step = 13
m = 0

list_ = ['WARNING', 'Recording onset was delayed and most words from the beginning are missing', 'The words retrieved may have inaccurate estimates of vocalization onset']
list2_ = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

for idx, n in enumerate(gt[0]):
  df = pd.read_csv(n, delimiter=";")
  #print(df)
  selected_rows = df[(~df['text'].isnull()) & (~df['Unnamed: 2'].isnull())]
  selected_rows = selected_rows[~selected_rows['text'].apply(lambda x: any(word in x for word in list2_))]
  selected_rows = selected_rows[~selected_rows['Unnamed: 2'].apply(lambda x: any(word in x for word in list_))]
  selected_rows = selected_rows[['text', 'Unnamed: 2']]

  i = 0
  j = 12
  trial = 1

  while j < 195:
    ground_truth = selected_rows[i:j]
    ground_truth = ground_truth[['text']]
    r = pd.read_csv(model_rec[0][m], delimiter=";")
    r = r[(r['from_trial'] == 'yes') & (r['trial_number'] == trial)]
    r = r[['word']]

    marks = []
    is_on_list = False

    for idx, t in ground_truth.iterrows():
      for jdx, tt in r.iterrows():
        if tt['word'].upper() == t['text']:
          is_on_list = True
      if is_on_list:
        marks.append(1)
        is_on_list = False
      else:
        marks.append(-1)

    new_table = selected_rows[i:j]
    new_table['MODEL'] = marks
    new_table.rename(columns = {'text':'WORD', 'Unnamed: 2':'MANUAL'}, inplace = True)
    new_table.loc[new_table["MODEL"] == -1, "MODEL"] = 0
    new_table["MANUAL"] = np.where(new_table["MANUAL"] == "-1", 0, 1)
    manual_model = check_comp(new_table)
    new_table['MANUAL_MODEL'] = manual_model
    new_table['TRIAL_NUMBER'] = trial
    new_table.reset_index(drop=True, inplace=True)
    
    path = recordings[m][0].split('words.csv')[0] + '/trials_check/trial_' + str(trial) + '.csv'
    filepath = Path(str(path))  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    new_table.to_csv(filepath)

    i = i + step
    j = j + step
    trial = trial + 1
  m = m + 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

## GENEROWANIE WYNIKÓW DLA SESJI

In [30]:
rootdir = os.fsencode('/content/recordings/download/')

for subdir, dirs, files in os.walk(rootdir):

  df_list = []

  if "trials_check" in str(subdir): 
    for file in os.listdir(subdir):
      filename = os.fsdecode(file)
      if filename.endswith(".csv"):
        #print(str(subdir.decode("utf-8")) + filename)
        df = pd.read_csv(str(subdir.decode("utf-8")) + '/' + filename)
        #df.drop(['Unnamed: 0'], axis=1)
        df_list.append(df)
        #recordings.append([os.path.join(str(subdir.decode("utf-8")), filename), i])
    new_df = pd.concat(df_list)
    new_df.rename(columns = {'Unnamed: 0':'INDEX'}, inplace = True)
    new_df = new_df.sort_values(by=['TRIAL_NUMBER', 'INDEX'])
    

    path = str(subdir.decode("utf-8")) + '/entire_session/session_sumup.csv'
    filepath = Path(str(path))  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    new_df.to_csv(filepath, index=False)

In [20]:
new_df

,INDEX,WORD,MANUAL,MODEL,MANUAL_MODEL,TRIAL_NUMBER
0,0,SAIL,1,0,0,1
1,1,ICE,1,0,0,1
2,2,SALT,0,0,1,1
3,3,FLEA,1,0,0,1
4,4,BIRD,0,0,1,1
...,...,...,...,...,...,...
175,7,MOTH,0,0,1,15
176,8,PANTS,1,0,0,15
177,9,BARN,1,0,0,15
178,10,CLOUD,0,0,1,15


## POBRANIE ZIPA Z WYNIKAMI

In [ ]:
!zip -r /content/recordings.zip /content/recordings

In [32]:
from google.colab import files
files.download("/content/recordings.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## TESTY

In [ ]:
#gt[0][0]
gt = pd.DataFrame.from_records(recordings)
df = pd.read_csv(gt[0][2], delimiter=";")
df = df[(df['from_trial'] == 'yes') & (df['trial_number'] == 2)]
df = df[['word']]

gt2 = pd.DataFrame.from_records(manual)
df2 = pd.read_csv(gt2[0][2], delimiter=";")[9:]

selected_rows = df2[(~df2['text'].isnull()) & (~df2['Unnamed: 2'].isnull())]
selected_rows = selected_rows[['text']]

marks = []
is_on_list = False

for idx, t in (selected_rows[13:25]).iterrows():
  #print(t['text'])
  for jdx, tt in df.iterrows():
    #print(tt['word'].upper())
    if tt['word'].upper() == t['text']:
      is_on_list = True
  if is_on_list:
    marks.append(1)
    is_on_list = False
  else:
    marks.append(-1)

marks

In [ ]:
gt = pd.DataFrame.from_records(manual)

#for idx, n in enumerate(gt[0]):
  #print(n)
df = pd.read_csv(gt[0][6], delimiter=";")
selected_rows = df[(~df['text'].isnull()) & (~df['Unnamed: 2'].isnull())]
list_ = ['WARNING', 'Recording onset was delayed and most words from the beginning are missing', 'The words retrieved may have inaccurate estimates of vocalization onset']
selected_rows = selected_rows[~selected_rows['Unnamed: 2'].apply(lambda x: any(word in x for word in list_))]
  #print(df['Unnamed: 2'])
#df[['Unnamed: 2']]
selected_rows

In [ ]:
gt = pd.DataFrame.from_records(manual)
df = pd.read_csv(gt[0][1], delimiter=";")[9:]

In [ ]:
df[~df['Unnamed: 2'].isnull()]

In [ ]:
selected_rows = df[(~df['text'].isnull()) & (~df['Unnamed: 2'].isnull())]
#selected_rows = selected_rows[~selected_rows['Unnamed: 2'].isnull()]
#selected_rows = selected_rows[~selected_rows['Unnamed: 2'] != '1 of 15 trials completed']
selected_rows = selected_rows[['text', 'Unnamed: 2']]
selected_rows

In [ ]:
selected_rows[26:38]

In [ ]:
i = 0
j = 12
step = 13
while(j != 90):
  print(i)
  print(j)
  i = i + step
  j = j + step